In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05052757263183594
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 5.7385079987166225
200000 10.999654746159205
300000 11.359479929124017
400000 12.367776369466165
500000 11.816796483793363
600000 12.791337094652986
700000 13.440588271897317
800000 16.491527499941924
900000 19.310086046105887
1000000 21.137056596695125
1100000 19.001846878208447
1200000 9.687444160540704
1300000 21.05226663099102
1400000 21.38890013812661
1500000 21.108438149263208
1600000 21.524410688763947
1700000 22.556218398786037
1800000 21.802415048929426
1900000 21.47001660024614
2000000 21.526823721728487
2100000 21.966957220335477
2200000 21.595707855674565
2300000 5.686284429323014
2400000 24.13944500060319
2500000 26.500549092652918
2600000 26.408761174796606
2700000 25.922428205667796
2800000 27.093704210505496
2900000 26.284239283889256
300000

29400000 23.04849447005367
29500000 7.874108216893405
29600000 23.44571378423105
29700000 22.979708177510556
29800000 21.61557534076804
29900000 22.783800837352825
30000000 21.10575723228441
30100000 21.210270058785895
30200000 20.693774801767706
30300000 20.283974253604846
30400000 5.877490099553882
30500000 20.224726536457478
30600000 20.708358170318615
30700000 20.560883307234118
30800000 21.37389542186121
30900000 21.506079194773314
31000000 21.221642313375106
31100000 22.403692357910263
31200000 21.697054892358416
31300000 9.206723459164321
31400000 21.963698930589867
31500000 21.837252866510372
31600000 22.13992665377723
31700000 18.672304170193964
31800000 11.144187608400621
31900000 21.43756126887076
32000000 21.9667740841818
32100000 21.91136415884348
32200000 20.405042693569598
32300000 20.5009178989479
32400000 20.460561725635888
32500000 19.7282187208198
32600000 12.65696721339515
32700000 13.98321611325417
32800000 19.767565650043093
32900000 20.075372647698643
33000000 20

59200000 17.328098282507298
59300000 16.158136441447006
59400000 17.320385703697234
59500000 15.777483483420353
59600000 7.07703244632826
59700000 15.40068984077263
59800000 16.676750375043788
59900000 16.883995486845386
60000000 15.978524025273323
60100000 18.069591998422442
60200000 17.039596944648387
60300000 7.457829236681381
60400000 18.732598315548735
60500000 18.28044262391136
60600000 19.109820855541788
60700000 17.60103535587281
60800000 19.468723871266572
60900000 17.663414625372518
61000000 17.988627913568436
61100000 18.22249250607816
61200000 8.440583753876671
61300000 16.724576690482724
61400000 16.060761680457837
61500000 16.718408441361166
61600000 16.191940658642768
61700000 15.910038068422589
61800000 13.78196630721647
61900000 8.474715784349979
62000000 16.322816595852423
62100000 15.353699143340805
62200000 15.911693926902414
62300000 16.58220447954692
62400000 16.620823564759462
62500000 9.238024209620376
62600000 15.72262244715224
62700000 18.32338666470948
628000

88900000 9.94576485815808
89000000 14.103852344585892
89100000 14.401737756823952
89200000 14.2488300397542
89300000 6.304711307520482
89400000 14.622574084385471
89500000 14.762498134238799
89600000 15.151047300398169
89700000 15.29449116045213
89800000 15.581399902395695
89900000 15.747579404232551
90000000 16.113340056274783
90100000 15.510397234642038
90200000 16.31425818188984
90300000 15.883980505435837
90400000 15.372717687304933
90500000 14.677499212219827
90600000 12.153050751317455
90700000 8.323471439570309
90800000 13.957592346861016
90900000 14.039533095143193
91000000 14.726041072168389
91100000 4.1070549184770515
91200000 15.035577758494036
91300000 14.425981798693012
91400000 14.34256779560752
91500000 7.979216812496691
91600000 12.93839336330214
91700000 14.725978130243837
91800000 15.325791547738268
91900000 15.613788656076938
92000000 15.702014912420811
92100000 15.9236394771785
92200000 15.893437089028726
92300000 15.888744143922096
92400000 15.902170179241509
92500

117900000 8.58394042056804
118000000 10.541888481198251
118100000 5.677390558822828
118200000 11.988915211347372
118300000 5.544595766130656
118400000 12.94541288136073
118500000 12.627002397666796
118600000 12.613535410301239
118700000 12.284446626027187
118800000 12.721618852056848
118900000 12.701099392653056
119000000 13.633687427534595
119100000 14.577495041568657
119200000 13.886670019703795
119300000 14.323239657398414
119400000 13.311396705219604
119500000 12.946360779818834
119600000 12.737030092810917
119700000 12.758721063192484
119800000 12.465181813623538
119900000 12.009056757436314
120000000 12.594831817034459
120100000 5.591185838585307
120200000 12.303208788441484
120300000 5.02620367795963
120400000 10.917246762121708
120500000 6.914868708452491
120600000 10.672843420756191
120700000 12.683916252394617
120800000 12.292358896099707
120900000 12.536507854740133
121000000 12.759428833838045
121100000 12.382614073650869
121200000 13.273874705320205
121300000 13.3693541044

...

175500000 9.41238822158541
175600000 9.527395284917654
175700000 8.606551719970986
175800000 8.812182697251082
175900000 8.651224838693876
176000000 8.56179925778784
176100000 8.676682293678926
176200000 8.407935168317199
176300000 3.6964097855278015
176400000 8.565557236109326
176500000 3.7111408224226534
176600000 7.494517597174672
176700000 5.883613304724232
176800000 6.371323442645432
176900000 8.502786909443822
177000000 8.608351323418452
177100000 8.528899304782719
177200000 8.672069759644437
177300000 8.942910684813006
177400000 8.688289341930757
177500000 9.62573600260545
177600000 9.389227863144372
177700000 9.355457832378507
177800000 9.036355404951232
177900000 8.889556753997137
178000000 8.563436843653857
178100000 8.456973359355535
178200000 8.602087051146727
178300000 8.109576118254324
178400000 8.261432187507632
178500000 5.135638566294187
178600000 7.053233990909139
178700000 6.780498727932977
178800000 4.020099475348105
178900000 8.101487369855493
179000000 3.605999856

204800000 7.263421751834217
204900000 6.79610792376619
205000000 6.8919515100846285
205100000 6.622400858826873
205200000 6.547114696211374
205300000 2.7137889328453935
205400000 6.606171932363289
205500000 6.283466778027699
205600000 6.454057089300702
205700000 4.282901278566586
205800000 4.450142509363145
205900000 6.370460185183644
206000000 6.527759180855841
206100000 6.424492454711553
206200000 2.439449354717741
206300000 6.493850699252824
206400000 6.6065857506709
206500000 6.76452230642718
206600000 6.971634904319733
206700000 7.3435621710681085
206800000 6.965827424727947
206900000 7.112529340472873
207000000 7.305468188882106
207100000 6.9748778546144266
207200000 6.836831691999237
207300000 6.810444010349162
207400000 6.509506525667786
207500000 3.9312950902589723
207600000 5.373076049138063
207700000 6.376920534523219
207800000 6.215335807043628
207900000 6.6422845464795
208000000 1.8126792025213343
208100000 6.604117212040603
208200000 6.295566639006549
208300000 6.17404294

234100000 5.345976724387327
234200000 4.898978731255813
234300000 1.9435546474723948
234400000 5.0270911056411265
234500000 4.703086435067584
234600000 4.629357061806424
234700000 4.44049522477135
234800000 4.6093012737229895
234900000 3.898043008168475
235000000 2.0373330327413184
235100000 4.551507415640334
235200000 4.393409272024194
235300000 4.559948478041679
235400000 4.464540693622384
235500000 4.829601188653587
235600000 3.1816508194150335
235700000 3.4054597041404677
235800000 5.095838720466342
235900000 4.9111265592489834
236000000 4.9953631834431915
236100000 4.857608390468333
236200000 4.882240418373674
236300000 5.043809542105638
236400000 4.884783080975591
236500000 2.8918820721074345
236600000 3.7126189577418303
236700000 4.712137316221096
236800000 4.357801516096453
236900000 4.3182232503341105
237000000 4.345431910016375
237100000 4.097480095611145
237200000 1.8850552415822877
237300000 3.8045782743161984
237400000 4.247877629862428
237500000 4.362016574157857
23760000

263000000 2.6090901490556764
263100000 2.786458094977319
263200000 2.530601752206981
263300000 1.0622617147771716
263400000 2.617107723438312
263500000 2.552947908120036
263600000 2.5417121415391466
263700000 2.486417476567063
263800000 2.539898496466279
263900000 2.287144905334453
264000000 2.366837029507031
264100000 2.274265445970635
264200000 0.6869545316513147
264300000 2.4293638172119225
264400000 2.154074010759652
264500000 2.441815661330405
264600000 2.3884860895644526
264700000 2.469185655275941
264800000 2.4475656715977556
264900000 2.3940359449910193
265000000 2.638093117980579
265100000 0.8965508008902735
265200000 2.4693494892962042
265300000 2.458630597738326
265400000 2.4611784722615218
265500000 1.5567379124442604
265600000 1.897569034903566
265700000 2.4842941542967822
265800000 2.415427102767683
265900000 2.3355080037863654
266000000 2.419047153051138
266100000 2.3172093645593193
266200000 2.3666907128333747
266300000 2.114760094727619
266400000 1.4436605467125707
266

291500000 0.4062402203856276
291600000 0.42402329280126433
291700000 0.4126049836938282
291800000 0.4327569203679959
291900000 0.4015777804975774
292000000 0.3993446081394652
292100000 0.4017901285695268
292200000 0.34887443957744707
292300000 0.0834262444050014
292400000 0.30720294228966033
292500000 0.3064100996176269
292600000 0.28843381639714044
292700000 0.28245719461326463
292800000 0.28424638769113353
292900000 0.2809545089599689
293000000 0.26458905425401863
293100000 0.2519766955580943
293200000 0.2490348178552687
293300000 0.23682150092919338
293400000 0.10643920849847462
293500000 0.20048892337071894
293600000 0.21942762928266987
293700000 0.21356465226626398
293800000 0.20239161980070672
293900000 0.19796689447877475
294000000 0.20260119763917722
294100000 0.1798752004183531
294200000 0.17924967750411233
294300000 0.1692938918740352
294400000 0.15671112080256144
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.10343340408341752
294600000 0.